# Single Leptoquark Analysis

In [ ]:
import os
import ROOT
from ROOT import *

In [ ]:
def getData (fullAnalysis=True):
    """
    Function to read data from Delphes-generated events
    """
    ev_dir = os.getcwd() + '/Events'    # Event directory path
    
    jobs = 1
    if fullAnalysis:
        root, dirs, files = os.walk(ev_dir)
        jobs = len(dirs)
    
    tree = TChain('Delphes;1')
    for i in range(jobs):
        path = str('Events/run_{:02d}/tag_1_delphes_events.root'.format(i))
        tree.Add(path)
        print('Run found in ' + path)
    print('Total events uploaded: {}'.format(tree.GetEntries()))

    return tree

In [ ]:
class JetVector:
    def __init__(self, event, j):
        self.TLV = TLorentzVector()
        self.TLV.SetPtEtaPhiM(event.GetLeaf('Jet.PT').GetValue(j),
                              event.GetLeaf('Jet.Eta').GetValue(j),
                              event.GetLeaf('Jet.Phi').GetValue(j),
                              event.GetLeaf('Jet.Mass').GetValue(j)
                             )
        self.BTag = event.GetLeaf('Jet.BTag').GetValue(j)
        self.TauTag = event.GetLeaf('Jet.TauTag').GetValue(j)
        
    
    ### Retrieval methods
        
    def GetTLV (self):    
        return self.TLV
    
    def GetBTag (self):
        return self.BTag
    
    def GetTauTag (self):
        return self.TauTag

    def PT(self):
        # Returns jet transverse momentum
        TLV = self.TLV
        return TLV.Pt()

    def Eta(self):
        # Returns jet pseudorapidity
        TLV = self.TLV
        return TLV.Eta()

    ### Delta methods

    def DeltaR(self, v2):
        TLV1 = self.TLV
        TLV2 = v2.GetTLV()
        return TLV1.DeltaR(TLV2)

    def DeltaEta(self, v2):
        TLV2 = v2.GetTLV()
        return self.Eta() - v2.Eta()

    def DeltaPhi(self, v2):
        TLV1 = self.TLV
        TLV2 = v2.GetTLV()
        return TLV1.DeltaPhi(TLV2)

    def sDeltaPT(self, v2):
        return self.PT() - v2.PT()

    def vDeltaPT(self, v2):
        TLV1 = self.TLV
        TLV2 = v2.GetTLV()
        a=TVector2(TLV1.Px(), TLV1.Py())
        b=TVector2(TLV2.Px(), TLV2.Py())
        c=a-b
        return c.Mod()

    def vDeltaP(self,v2):
        TLV1 = self.TLV
        TLV2 = v2.GetTLV()
        a=TVector3(TLV1.Px(), TLV1.Py(), TLV1.Pz())
        b=TVector3(TLV2.Px(), TLV2.Py(), TLV2.Pz())
        c=a-b
        return c.Mag()

In [ ]:
def PT(jet):
    # Returns jet transverse momentum
    return jet.PT()

def getGoodJets(event):
    """
    Separate jet types into a map structure.
    """
    jetTypes = {'jets' : [],
                'b_jets' : [],
                'tau_jets' : [],
                'all_jets' : [],
                'other_jets' : []
               }
    
    n_jets = event.Jet.GetEntries()
    
    for j in range(n_jets):
        jet = JetVector(event, j)
        
        if PT(jet)<= 20.:
            continue
        
        jetTypes['all_jets'] += [jet]
        if (jet.GetBTag() == 0 and jet.GetBTag() == 0):
            if jet.PT()>30.:
                jetTypes['jets'] += [jet]
                
        elif (jet.GetBTag() == 1 and jet.GetBTag() == 0):
            if (jet.PT()>30.) and abs(jet.Eta())<2.5:
                jetTypes['b_jets'] += [jet]
        
        elif (jet.GetBTag() == 0 and jet.GetBTag() == 1):
            if abs(jet.Eta())<2.5:
                jetTypes['tau_jets'] += [jet]
        else:
            jetTypes['other_jets'] += [jet]
            
    for key in jetTypes:
        jetTypes[key].sort(reverse = True, key = PT)
        
    return jetTypes

In [ ]:
canvas = TCanvas("Canvas","",800,600)
canvas.SetGrid()
cutflow = TH1F("Cutflow","Cutflow; Cut; Events",10,0,10)
hist_njets = TH1F("Number of jets","n-jets; Jet multiplicity; Events",10,0,10)
hist_m_LQ=TH1F("hist_m_LQ", "M_{b tau}", 1000, 0.0, 2000.0)

pt_all_jets=TH1F("pt_all_jets", "Pt_{allj}", 100, 0.0, 600.0)
eta_all_jets=TH1F("eta_all_jets", "#eta_{allj}", 100, -5, 5)
pt_all_jets.SetLineColor(kBlack)
eta_all_jets.SetLineColor(kBlack)

pt_lead_jets=TH1F("pt_lead_jets", "Pt_{j1}", 100, 0.0, 1000.0)
eta_lead_jets=TH1F("eta_lead_jets", "#eta_{j1}", 100, -5, 5)
pt_lead_jets.SetLineColor(kBlue)
eta_lead_jets.SetLineColor(kBlue)
pt_lead_bjets=TH1F("pt_lead_bjets", "Pt_{bj1}", 100, 0.0, 1000.0)
eta_lead_bjets=TH1F("eta_lead_bjets", "#eta_{bj2}", 100, -5, 5)
pt_lead_bjets.SetLineColor(kRed)
eta_lead_bjets.SetLineColor(kRed)
pt_lead_taus=TH1F("pt_lead_taus", "Pt_{tau1}", 100, 0.0, 1000.0)
eta_lead_taus=TH1F("eta_lead_taus", "#eta_tau1", 100, -5, 5)
pt_lead_taus.SetLineColor(kGreen)
eta_lead_taus.SetLineColor(kGreen)
pt_slead_taus=TH1F("pt_slead_taus", "Pt_tau2", 100, 0.0, 1000.0)
eta_slead_taus=TH1F("eta_slead_taus", "#eta_tau2", 100, -5, 5)
pt_slead_taus.SetLineColor(kViolet)
eta_slead_taus.SetLineColor(kViolet)

hist_deltar_taus=TH1F("deta_taus","#Delta R_{#tau_{1}#tau_{2}}",100,0,8) 
hist_deltar_b_ltau=TH1F("deltar_b_ltau","#Delta R_{b#tau_{1}}",100,0,8) 
hist_deltar_b_sltau=TH1F("deltar_b_sltau","#Delta R_{b#tau_{2}}",100,0,8) 
hist_deltar_j_ltau=TH1F("deltar_j_ltau","#Delta R_{j#tau_{1}}",100,0,8) 
hist_deltar_j_sltau=TH1F("deltar_j_sltau","#Delta R_{j#tau_{2}}",100,0,8) 
hist_deltar_b_j=TH1F("deltar_b_j","#Delta R_{bj}",100,0,8)

hist_deltaEta_taus=TH1F("deltaEta_taus","#Delta #eta_{#tau_{1}#tau_{2}}",100,-5,5)
hist_deltaEta_b_ltau=TH1F("deltaEta_b_ltau","#Delta #eta_{b#tau_{1}}",100,-5,5) 
hist_deltaEta_b_sltau=TH1F("deltaEta_b_sltau","#Delta #eta_{b#tau_{2}}",100,-5,5) 
hist_deltaEta_j_ltau=TH1F("deltaEta_j_ltau","#Delta #eta_{j#tau_{1}}",100,-5,5) 
hist_deltaEta_j_sltau=TH1F("deltaEta_j_sltau","#Delta #eta_{j#tau_{2}}",100,-5,5) 
hist_deltaEta_b_j=TH1F("deltaEta_b_j","#Delta #eta_{bj}",100,-5,5)

hist_deltaPhi_taus=TH1F("deltaPhi_taus","#Delta #phi_{#tau_{1}#tau_{2}}",100,-5,5) 
hist_deltaPhi_b_ltau=TH1F("deltaPhi_b_ltau","#Delta #phi_{b#tau_{1}}",100,-5,5) 
hist_deltaPhi_b_sltau=TH1F("deltaPhi_b_sltau","#Delta #phi_{b#tau_{2}}",100,-5,5) 
hist_deltaPhi_j_ltau=TH1F("deltaPhi_j_ltau","#Delta #phi_{j#tau_{1}}",100,-5,5) 
hist_deltaPhi_j_sltau=TH1F("deltaPhi_j_sltau","#Delta #phi_{j#tau_{2}}",100,-5,5) 
hist_deltaPhi_b_j=TH1F("deltaPhi_b_j","#Delta #phi_{bj}",100,-5,5)

hist_sdPT_taus=TH1F("sdPT_taus","#Delta PT_{#tau_{1}#tau_{2}}",100,0,1000) 
hist_sdPT_b_ltau=TH1F("sdPT_b_ltau","#Delta PT_{b#tau_{1}}",100,-800,800) 
hist_sdPT_b_sltau=TH1F("sdPT_b_sltau","#Delta PT_{b#tau_{2}}",100,-800,800) 
hist_sdPT_j_ltau=TH1F("sdPT_j_ltau","#Delta PT_{j#tau_{1}}",100,-800,800) 
hist_sdPT_j_sltau=TH1F("sdPT_j_sltau","#Delta PT_{j#tau_{2}}",100,-800,800) 
hist_sdPT_b_j=TH1F("sdPT_b_j","#Delta PT_{bj}",100,-800,800)

hist_vdPT_taus=TH1F("vdPT_taus","#Delta #vec{PT}_{#tau_{1}#tau_{2}}",100,0,1000) 
hist_vdPT_b_ltau=TH1F("vdPT_b_ltau","#Delta #vec{PT}_{b#tau_{1}}",100,0,800) 
hist_vdPT_b_sltau=TH1F("vdPT_b_sltau","#Delta #vec{PT}_{b#tau_{2}}",100,0,800) 
hist_vdPT_j_ltau=TH1F("vdPT_j_ltau","#Delta #vec{PT}_{j#tau_{1}}",100,0,800) 
hist_vdPT_j_sltau=TH1F("vdPT_j_sltau","#Delta #vec{PT}_{j#tau_{2}}",100,0,800) 
hist_vdPT_b_j=TH1F("vdPT_b_j","#Delta #vec{PT}_{bj}",100,0,800)

hist_vdP_taus=TH1F("vdP_taus","#Delta #vec{P}_{#tau_{1}#tau_{2}}",100,0,1000) 
hist_vdP_b_ltau=TH1F("vdP_b_ltau","#Delta #vec{P}_{b#tau_{1}}",100,0,800) 
hist_vdP_b_sltau=TH1F("vdP_b_sltau","#Delta #vec{P}_{b#tau_{2}}",100,0,800) 
hist_vdP_j_ltau=TH1F("vdP_j_ltau","#Delta #vec{P}_{j#tau_{1}}",100,0,800) 
hist_vdP_j_sltau=TH1F("vdP_j_sltau","#Delta #vec{P}_{j#tau_{2}}",100,0,800) 
hist_vdP_b_j=TH1F("vdP_b_j","#Delta #vec{P}_{bj}",100,0,800)

pt_forward_jets=TH1F("pt_forward_jets", "Pt_{fj}", 100, 0.0, 600.0)
pt_central_jets=TH1F("pt_central_jets", "Pt_{cj}", 100, 0.0, 600.0)

In [ ]:
def FillHistos(jetDict):
    pt_lead_jets.Fill  (jetDict['jets'][0].PT() )
    eta_lead_jets.Fill (jetDict['jets'][0].Eta())
    pt_lead_bjets.Fill (jetDict['b_jets'][0].Pt() )
    eta_lead_bjets.Fill(jetDict['b_jets'][0].Eta())
    pt_lead_taus.Fill  (jetDict['tau_jets'][0].Pt() )
    eta_lead_taus.Fill (jetDict['tau_jets'][0].Eta())
    pt_slead_taus.Fill (jetDict['tau_jets'][1].Pt() )
    eta_slead_taus.Fill(jetDict['tau_jets'][1].TLV.Eta())
    
    pt_all_jets.Fill (jetDict['b_jets'][0].Pt() )
    eta_all_jets.Fill(jetDict['b_jets'][0].Eta())
    pt_all_jets.Fill (jetDict['tau_jets'][0].Pt()  )
    eta_all_jets.Fill(jetDict['tau_jets'][0].Eta() )
    pt_all_jets.Fill (jetDict['tau_jets'][1].Pt()  )
    eta_all_jets.Fill(jetDict['tau_jets'][1].Eta() )

    pt_all_jets.Fill (jetDict['jets'][0].Pt() )
    eta_all_jets.Fill(jetDict['jets'][0].Eta())
    hist_deltar_taus.Fill(DeltaR(taus[0],taus[1]))
    hist_deltar_b_ltau.Fill(DeltaR(bjets[0],taus[0]))
    hist_deltar_b_sltau.Fill(DeltaR(bjets[0],taus[1]))
    hist_deltar_j_ltau.Fill(DeltaR(jets[0],taus[0]))
    hist_deltar_j_sltau.Fill(DeltaR(jets[0],taus[1]))
    hist_deltar_b_j.Fill(DeltaR(bjets[0],jets[0]))

    hist_deltaEta_taus.Fill(DeltaEta(taus[0],taus[1]))
    hist_deltaEta_b_ltau.Fill(DeltaEta(bjets[0],taus[0]))
    hist_deltaEta_b_sltau.Fill(DeltaEta(bjets[0],taus[1]))
    hist_deltaEta_j_ltau.Fill(DeltaEta(jets[0],taus[0]))
    hist_deltaEta_j_sltau.Fill(DeltaEta(jets[0],taus[1]))
    hist_deltaEta_b_j.Fill(DeltaEta(bjets[0],jets[0]))

    hist_deltaPhi_taus.Fill(DeltaPhi(taus[0],taus[1]))
    hist_deltaPhi_b_ltau.Fill(DeltaPhi(bjets[0],taus[0]))
    hist_deltaPhi_b_sltau.Fill(DeltaPhi(bjets[0],taus[1]))
    hist_deltaPhi_j_ltau.Fill(DeltaPhi(jets[0],taus[0]))
    hist_deltaPhi_j_sltau.Fill(DeltaPhi(jets[0],taus[1]))
    hist_deltaPhi_b_j.Fill(DeltaPhi(bjets[0],jets[0]))

    hist_sdPT_taus.Fill(sDeltaPT(taus[0],taus[1]))
    hist_sdPT_b_ltau.Fill(sDeltaPT(bjets[0],taus[0]))
    hist_sdPT_b_sltau.Fill(sDeltaPT(bjets[0],taus[1]))
    hist_sdPT_j_ltau.Fill(sDeltaPT(jets[0],taus[0]))
    hist_sdPT_j_sltau.Fill(sDeltaPT(jets[0],taus[1]))
    hist_sdPT_b_j.Fill(sDeltaPT(bjets[0],jets[0]))
    
    hist_vdPT_taus.Fill(vDeltaPT(taus[0],taus[1]))
    hist_vdPT_b_ltau.Fill(vDeltaPT(bjets[0],taus[0]))
    hist_vdPT_b_sltau.Fill(vDeltaPT(bjets[0],taus[1]))
    hist_vdPT_j_ltau.Fill(vDeltaPT(jets[0],taus[0]))
    hist_vdPT_j_sltau.Fill(vDeltaPT(jets[0],taus[1]))
    hist_vdPT_b_j.Fill(vDeltaPT(bjets[0],jets[0]))
        
    hist_vdP_taus.Fill(vDeltaP(taus[0],taus[1]))
    hist_vdP_b_ltau.Fill(vDeltaP(bjets[0],taus[0]))
    hist_vdP_b_sltau.Fill(vDeltaP(bjets[0],taus[1]))
    hist_vdP_j_ltau.Fill(vDeltaP(jets[0],taus[0]))
    hist_vdP_j_sltau.Fill(vDeltaP(jets[0],taus[1]))
    hist_vdP_b_j.Fill(vDeltaP(bjets[0],jets[0]))
    
    P=bjets[0].TLV + taus[0].TLV
    hist_m_LQ.Fill(TMath.Sqrt(P*P))

In [ ]:
for event in tree:
    
    n_jets=event.Jet.GetEntries()
    hist_njets.Fill(n_jets)
    cutflow.Fill(0)
    
    if not (n_jets>0): continue
    cutflow.Fill(1)
    
    if not (n_jets>1): continue
    cutflow.Fill(2)
    
    if not (n_jets>2): continue
    cutflow.Fill(3)
    
    if not (n_jets>3): continue
    cutflow.Fill(4)
    
    jetTypes = getGoodJets(event)
    
    if not (len(jetTypes['tau_jets'])>0): continue
    cutflow.Fill(5)
    
    if not (len(jetTypes['tau_jets'])>1): continue
    cutflow.Fill(6)
 
    if not (len(jetTypes['b_jets'])>0): continue
    cutflow.Fill(7)
    
    if not (len(jetTypes['jets'])>0): continue
    cutflow.Fill(8)
    
    FillHistos(jetTypes)
    
    if TMath.Abs(jets[0].TLV.Eta()>=2.5):
        pt_forward_jets.Fill(jets[0].TLV.Pt())
    else:
        pt_central_jets.Fill(jets[0].TLV.Pt())